# Отправка крипты на кошельки с OKX и отправка с кошельков на саб-аккаунты

* [Варианты подключения](#1)
* [-- Вариант 1 - Подключение с помощью requests](#2)
* [-- Вариант 2 - Подключение с помощью библиотеки Okx](#3)


* [Часть 1 - Пополнение кошельков с Okx](#4)
* [-- Вариант 1 - отправить один вид монет в равном количестве на кошельки](#5)
* [-- Вариант 2 - настраиваемая отправка: разные монеты в разном количестве на кошельки](#6)


* [Часть 2 - отправка крипты с кошельков на саб-аккаунты Okx](#7)
* [-- Вариант 1 - отправка нативных монет](#8)
* [-- Вариант 2 - отправка токенов, например стейблов](#9)



Создайте ключ в личном кабинете.

Вы получите api-ключ, секретный ключ, также вам понадобится серкретная фраза, которая использовалась для создания этих ключей.

In [1]:
API_KEY = '8gb2d2f2-274b-455d-967e-9fdcb490f46c'
SECRET_KEY = 'D27A0AA2C55BC18B15055580921E56209'
PASSPHRASE = 'secret_phrase'

## Варианты подключения<a class="anchor" id="1"></a>

### Вариант 1 - Подключение с помощью requests<a class="anchor" id="2"></a>

Документация: https://www.okx.com/docs-v5/en/?python#overview-rest-authentication<br>
Код взят отсюда: https://stackoverflow.com/questions/66486374/how-to-sign-an-okex-api-request

#### В качестве примера подключения ниже представлен код проверки баланса на Основном аккаунте

Необходимые библиотеки:

In [2]:
import requests
import json
import datetime

import hmac
import base64

Настройки:

In [3]:
OKX_DOMAIN = 'https://www.okx.com'

Ресурс для проверки баланса:

In [4]:
ENDPOINT = '/api/v5/asset/balances'

Функция для отправки даты и времени в формате UTC, например 2020-12-08T09:08:57.715Z

In [5]:
def get_time():
    now = datetime.datetime.utcnow()
    timestamp = now.isoformat("T", "milliseconds")
    return timestamp + "Z"

Функция для создания подписи в нужном формате, указанном в документации:

In [6]:
def signature(timestamp, method, endpoint, body, secret_key):
    if str(body) == '{}' or str(body) == 'None':
        body = ''
    message = str(timestamp) + str.upper(method) + endpoint + str(body)
    mac = hmac.new(bytes(secret_key, encoding='utf8'), bytes(message, encoding='utf-8'), digestmod='sha256')
    d = mac.digest()
    return base64.b64encode(d)

Функция для создания заголовка:

In [7]:
def get_header():
    body= {}
    method= 'GET'
    header = dict()
    header['CONTENT-TYPE'] = 'application/json'
    header['OK-ACCESS-KEY'] = API_KEY
    header['OK-ACCESS-SIGN'] = signature(get_time(), method, ENDPOINT, body, SECRET_KEY)
    header['OK-ACCESS-TIMESTAMP'] = str(get_time())
    header['OK-ACCESS-PASSPHRASE'] = PASSPHRASE
    return header

#### Подключение:

In [8]:
url = OKX_DOMAIN + ENDPOINT
header = get_header()
response= requests.get(url, headers=header)

Результат:

In [9]:
result = response.json()

In [10]:
result['data']

[{'availBal': '0.091180110676703',
  'bal': '0.091180110676703',
  'ccy': 'USDT',
  'frozenBal': '0'},
 {'availBal': '222.56312321576398',
  'bal': '222.56312321576398',
  'ccy': 'USDC',
  'frozenBal': '0'},
 {'availBal': '0.0000000098951581',
  'bal': '0.0000000098951581',
  'ccy': 'ETH',
  'frozenBal': '0'},
 {'availBal': '0.00064127',
  'bal': '0.00064127',
  'ccy': 'ALGO',
  'frozenBal': '0'},
 {'availBal': '0.0000000087097375',
  'bal': '0.0000000087097375',
  'ccy': 'AVAX',
  'frozenBal': '0'},
 {'availBal': '0.0000000095537055',
  'bal': '0.0000000095537055',
  'ccy': 'BNB',
  'frozenBal': '0'},
 {'availBal': '0.00063864',
  'bal': '0.00063864',
  'ccy': 'DOT',
  'frozenBal': '0'},
 {'availBal': '0.000000000670785',
  'bal': '0.000000000670785',
  'ccy': 'FTM',
  'frozenBal': '0'},
 {'availBal': '0.0000000036571',
  'bal': '0.0000000036571',
  'ccy': 'GODS',
  'frozenBal': '0'},
 {'availBal': '0.00001182558834',
  'bal': '0.00001182558834',
  'ccy': 'IMX',
  'frozenBal': '0'},
 

#### Проверка по конкретному ассету

Нужно изменить адрес ресурса - добавить параметр ccy с тикером ассета:

In [11]:
ENDPOINT = '/api/v5/asset/balances?ccy=USDC'

In [12]:
url = OKX_DOMAIN + ENDPOINT
header = get_header()
response= requests.get(url, headers=header)

In [13]:
result = response.json()

In [14]:
result['data']

[{'availBal': '222.56312321576398',
  'bal': '222.56312321576398',
  'ccy': 'USDC',
  'frozenBal': '0'}]

Баланс:

In [15]:
result['data'][0]['bal']

'222.56312321576398'

### Вариант 2 - Подключение с помощью библиотеки Okx<a class="anchor" id="3"></a>

Okx предлагает библиотеку https://pypi.org/project/python-okx/, которая сокращает количество кода и делает работу немного удобнее.

In [ ]:
pip install python-okx

#### С помощью библиотеки okx выполним аналогичную задачу, которая выше решалась с помощью requests

In [16]:
import okx.Funding as Funding

Подключение. Необходимо передать api-ключ, секретный ключ и секретную фразу. Еще в документации присутствует bool параметр False, не разобрался зачем он нужен:) И параметр 0 - реальный аккаунт или 1 - демо аккаунт.

In [17]:
funding_api = Funding.FundingAPI(API_KEY, SECRET_KEY, PASSPHRASE, False, '0')

In [20]:
result = funding_api.get_balances()

In [19]:
result['data']

[{'availBal': '0.091180110676703',
  'bal': '0.091180110676703',
  'ccy': 'USDT',
  'frozenBal': '0'},
 {'availBal': '222.56312321576398',
  'bal': '222.56312321576398',
  'ccy': 'USDC',
  'frozenBal': '0'},
 {'availBal': '0.0000000098951581',
  'bal': '0.0000000098951581',
  'ccy': 'ETH',
  'frozenBal': '0'},
 {'availBal': '0.00064127',
  'bal': '0.00064127',
  'ccy': 'ALGO',
  'frozenBal': '0'},
 {'availBal': '0.0000000087097375',
  'bal': '0.0000000087097375',
  'ccy': 'AVAX',
  'frozenBal': '0'},
 {'availBal': '0.0000000095537055',
  'bal': '0.0000000095537055',
  'ccy': 'BNB',
  'frozenBal': '0'},
 {'availBal': '0.00063864',
  'bal': '0.00063864',
  'ccy': 'DOT',
  'frozenBal': '0'},
 {'availBal': '0.000000000670785',
  'bal': '0.000000000670785',
  'ccy': 'FTM',
  'frozenBal': '0'},
 {'availBal': '0.0000000036571',
  'bal': '0.0000000036571',
  'ccy': 'GODS',
  'frozenBal': '0'},
 {'availBal': '0.00001182558834',
  'bal': '0.00001182558834',
  'ccy': 'IMX',
  'frozenBal': '0'},
 

#### Проверка по конкретному ассету

Нужно передать в качестве параметра тикер валюты:

In [23]:
result = funding_api.get_balances('USDC')

In [22]:
result['data']

[{'availBal': '222.56312321576398',
  'bal': '222.56312321576398',
  'ccy': 'USDC',
  'frozenBal': '0'}]

Баланс:

In [24]:
result['data'][0]['bal']

'222.56312321576398'

#### Все дальнейшие варианты пополнения кошельков через биржу okx буду делать на этой библиотеке

## Часть 1 - Пополнение кошельков с Okx<a class="anchor" id="4"></a>

### Подготовка

1. Необходимо добавить адреса для вывода через личный кабинет. Не нашел способ добавить адреса по API :(<br>
Инструкция https://www.okx.com/ru/help/10270095472397<br>
Если вы будете добавлять EVM адреса, чтобы в будущем отправлять на любые emv-сети любые активы, при добавлении адреса выбирайте Тип адреса - Адрес EVM. Поставьте галочку "Сохранить адрес как проверенный..."

2. Для отправки монет на кошельки нужно знать обозначение сети, в которую будет отправлен актив и значение комиссии.<br>
Как называется сеть можно узнать через личный кабинет - вывод средств<br>
<br>
Или получить всю информацию по API:

Файл get-assets-info.py

In [25]:
import pandas as pd

import okx.Funding as Funding

Настройки вашего аккаунта:

In [26]:
API_KEY = '8gb2d2f2-274b-455d-967e-9fdcb490f46c'
SECRET_KEY = 'D27A0AA2C55BC18B15055580921E56209'
PASSPHRASE = 'secret_phrase'

Подключение:

In [27]:
fundingAPI = Funding.FundingAPI(API_KEY, SECRET_KEY, PASSPHRASE, False, '0')

Получение информации об ассетах. <br>
Документация https://www.okx.com/docs-v5/en/?python#funding-account-rest-api-get-currencies

In [30]:
result = fundingAPI.get_currencies()

In [29]:
result['data']

[{'canDep': True,
  'canInternal': True,
  'canWd': False,
  'ccy': 'USDT',
  'chain': 'USDT-OKTC',
  'depQuotaFixed': '',
  'depQuoteDailyLayer2': '',
  'logoLink': 'https://static.coinall.ltd/cdn/oksupport/asset/currency/icon/usdt20230419113051.png',
  'mainNet': False,
  'maxFee': '0',
  'maxFeeForCtAddr': '0',
  'maxWd': '13400900',
  'minDep': '0.00000001',
  'minDepArrivalConfirm': '2',
  'minFee': '0',
  'minFeeForCtAddr': '0',
  'minWd': '0',
  'minWdUnlockConfirm': '4',
  'name': 'Tether',
  'needTag': False,
  'usedDepQuotaFixed': '',
  'usedWdQuota': '64.68',
  'wdQuota': '10000000',
  'wdTickSz': '8'},
 {'canDep': True,
  'canInternal': True,
  'canWd': True,
  'ccy': 'USDT',
  'chain': 'USDT-ERC20',
  'depQuotaFixed': '',
  'depQuoteDailyLayer2': '',
  'logoLink': 'https://static.coinall.ltd/cdn/oksupport/asset/currency/icon/usdt20230419113051.png',
  'mainNet': False,
  'maxFee': '7.64',
  'maxFeeForCtAddr': '7.64',
  'maxWd': '13400900',
  'minDep': '0.00000001',
  'minD

Запись в список со словарями название монет, названия сетей, в которые можно отправить монету, минимальную комиссию и максимальную:

In [ ]:
list_of_dicts = []
for element in range(len(result['data'])):
    dictionary = dict()
    dictionary['asset_name'] = result['data'][element]['ccy']
    dictionary['chain_name'] = result['data'][element]['chain']
    dictionary['min_fee'] = result['data'][element]['minFee']
    dictionary['max_fee'] = result['data'][element]['maxFee']
    list_of_dicts.append(dictionary)

Запись в датафрейм:

In [ ]:
df = pd.DataFrame.from_dict(list_of_dicts)

In [ ]:
df

Экспорт в Excel:

In [ ]:
writer_kernel = pd.ExcelWriter('assets_settings.xlsx', engine='xlsxwriter')
df.to_excel(writer_kernel, index=False)
writer_kernel.close()

### Вариант 1 - отправить один вид монет в равном количестве на кошельки <a class="anchor" id="5"></a>

Файл withdraw-from-okx-to-wallets-var-1.py

Этот вариант скрипта получает список адресов и распыляет общую сумму, указанную в TOTAL_SENDING_AMOUNT, на все кошельки. Также скрипт немного уникализирует каждую транзакцию, создавая уникальное количество монет к отправке и разное время задержки между транзакциями.

#### Библиотеки

In [31]:
import random
import time

import pandas as pd

import okx.Funding as Funding

#### Настройки

Настройки вашего аккаунта:

In [32]:
API_KEY = '8gb2d2f2-274b-455d-967e-9fdcb490f46c'
SECRET_KEY = 'D27A0AA2C55BC18B15055580921E56209'
PASSPHRASE = 'secret_phrase'

Название файла с настройками:

Excel:

In [59]:
SETTINGS_XLSX = 'wallet_addresses_var_1.xlsx'

или TXT:

In [60]:
SETTINGS_TXT = 'wallet_addresses_var_1.txt'

Тикер:

In [33]:
ASSET_NAME = 'USDC'

Название сети:

In [34]:
CHAIN_NAME = 'USDC-Arbitrum One (Bridged)'

Комиссия, использую значение minFee:

In [35]:
FEE = 0.1

Общее количество актива, которое будет раскидано на все кошельки:

In [36]:
TOTAL_SENDING_AMOUNT = 50

#### Дополнительные настройки

Стоит попытаться немного уникализировать тразакции, можно повлиять на отправляемую сумму и время - задержку между отправками.

Для задержки между транзакциями нужно задать минимальное время задержки и максимальное. Из диапазона этих значений будет выбираться случайное число.

Минимальная задержка (сек):

In [37]:
MIN_TIMESLEEP_BETWEEN_TRANSACTIONS = 240

Максимальная (сек):

In [38]:
MAX_TIMESLEEP_BETWEEN_TRANSACTIONS = 480

Чтобы уникализировать сумму отправки и вписаться в отправляемый бюджет, буду генерировать случайное число и отнимать его от отправляемой суммы. Это число будет отпределяться с помощью функции random.uniform(min_value, max_value), которая будет выдавать случайное число с плавающей точкой. Диапазон будет состоять из минимального значения TOTAL_SENDING_AMOUNT, деленный на MIN_VALUE_IN_RANGE_DIVIDER и максимального - TOTAL_SENDING_AMOUNT, деленный на MAX_VALUE_IN_RANGE_DIVIDER.

In [39]:
MIN_VALUE_IN_RANGE_DIVIDER = 10000

In [40]:
MAX_VALUE_IN_RANGE_DIVIDER = 100

#### Прежде, чем отправить крипту на кошельки, можете воспользоваться скриптом для предварительного расчета наобходимой суммы на балансе

Файл count-withdrawal-cost-from-okx.py

Количество кошельков:

In [120]:
WALLETS_QUANTITY = 10

Например, я собираюсь разбросать по 10 кошелькам (WALLETS_QUANTITY) 50 USDC (TOTAL_SENDING_AMOUNT). 

Отправляемое количество крипты для каждого кошелька будет определяться так:

общая сумма для всех кошельков делится на количество кошельков, вычитается комиссия, вычитается случайное число.

In [42]:
TOTAL_SENDING_AMOUNT / WALLETS_QUANTITY - FEE - random.uniform(TOTAL_SENDING_AMOUNT / MIN_VALUE_IN_RANGE_DIVIDER,
                                                               TOTAL_SENDING_AMOUNT / MAX_VALUE_IN_RANGE_DIVIDER)

4.499100741845885

In [43]:
total = []
for number in range(10):
    sent_amount = TOTAL_SENDING_AMOUNT / WALLETS_QUANTITY - FEE - random.uniform(
        TOTAL_SENDING_AMOUNT / MIN_VALUE_IN_RANGE_DIVIDER,
        TOTAL_SENDING_AMOUNT / MAX_VALUE_IN_RANGE_DIVIDER
    )
    
    print('Wallet №:', number, 'sent:', sent_amount)
    
    total.append(sent_amount)

Wallet №: 0 sent: 4.812253779235783
Wallet №: 1 sent: 4.612221727271949
Wallet №: 2 sent: 4.668570105343889
Wallet №: 3 sent: 4.830160496508188
Wallet №: 4 sent: 4.709376806659018
Wallet №: 5 sent: 4.633230237130149
Wallet №: 6 sent: 4.858001741572826
Wallet №: 7 sent: 4.661270977081915
Wallet №: 8 sent: 4.4153703382378575
Wallet №: 9 sent: 4.696313376050401


Итоговая отправленная сумма немного меньше TOTAL_SENDING_AMOUNT:

In [44]:
sum(total)

46.896769585091974

На комиссии ушло:

In [45]:
FEE * WALLETS_QUANTITY

1.0

#### Таким образом можно предварительно посмотреть, сколько крипты будет выведено на кошельки при определенных настройках, и сколько нужно крипты на балансе биржи, чтобы хватило на все с учетом комиссий.

Название файла с отчетом xlsx, который используется в функции ниже:

In [46]:
REPORT_NAME = 'report_table_var_1.xlsx'

#### Функции для создания отчета

Дополнительно напишу функцию, которая будет создавать отчет и сохраняться его в xlsx. В нем будет записан адрес кошелька, отправленная сумма, поле с кодом и расшифровкой ошибки, если транзакция будет отклонена.

excel_export записывает данные в excel. Принивает таблицу с данными (датафрейм), название будущего отчета:

In [47]:
def excel_export(table, table_name):
    writer_kernel = pd.ExcelWriter(table_name, engine='xlsxwriter')
    table.to_excel(writer_kernel, index=False)
    writer_kernel.close()

open_table открывает таблицу с отчетом, если таблицы нет, создает ее:

In [48]:
def open_table():
    try:
        table = pd.read_excel(REPORT_NAME)
        
        return table.to_dict('records')
    
    except:
        table_template = pd.DataFrame(index=[0])
        excel_export(table_template, REPORT_NAME)
        created_table = pd.read_excel(REPORT_NAME)
      
        return created_table.to_dict('records')

create_report принимает открытую таблицу, адрес кошелька, название монеты, сеть, отправленную сумму, ошибку, если она возникла, затем записывает данные в соответствующие поля, экспортирует xlsx:

In [49]:
def create_report(report_table, address, asset_name, chain, sent_amount, error):
        
    report_dict = dict()
    report_dict['address'] = address
    report_dict['asset_name'] = asset_name
    report_dict['chain'] = chain
    report_dict['sent_amount'] = sent_amount
    report_dict['error'] = error
        
    report_table.append(report_dict)
        
    result_table = pd.DataFrame.from_dict(report_table)
    
    result_table.dropna(inplace=True)
    
    excel_export(result_table, REPORT_NAME)

#### Загрузка адресов кошельков

Excel:

In [50]:
wallet_addresses_xlsx = pd.read_excel(SETTINGS_XLSX)

Загрузилась таблица со списком адресов в поле с заголовком address:

In [51]:
wallet_addresses_xlsx

,address
0,0xA3D2e233658387701B92f0DA0cD422E314490C1F
1,0xf02EF849af18e6BBa06024F8Ed4C56A2c5ea1556
2,0x1902b720961c1E257d4A290726A5E8809903de0D
3,0x7F29bAeE91eadbF7033b1B55536a063edD7242a8
4,0xe6A1faF76e7b8281F66173e506AC7341f707FEBF
5,0xc9524939551d5C97d611029A3C8150694396F537
6,0x60378c4E2e56b7790839DA521E777B78aB1bfB5d
7,0x065968Cfcf80d99b9695F36BdBa90B67B0B48505
8,0x3309821aa89b1D42fe8EACff2FA56CC8c5FA06FE
9,0xdf24026b19c60fCF9cD6145c237BD140EFe9f390


Конвертация серии датафрейма в list:

In [52]:
wallet_addresses_list = wallet_addresses_xlsx['address'].to_list()

In [53]:
wallet_addresses_list

['0xA3D2e233658387701B92f0DA0cD422E314490C1F',
 '0xf02EF849af18e6BBa06024F8Ed4C56A2c5ea1556',
 '0x1902b720961c1E257d4A290726A5E8809903de0D',
 '0x7F29bAeE91eadbF7033b1B55536a063edD7242a8',
 '0xe6A1faF76e7b8281F66173e506AC7341f707FEBF',
 '0xc9524939551d5C97d611029A3C8150694396F537',
 '0x60378c4E2e56b7790839DA521E777B78aB1bfB5d',
 '0x065968Cfcf80d99b9695F36BdBa90B67B0B48505',
 '0x3309821aa89b1D42fe8EACff2FA56CC8c5FA06FE',
 '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390']

TXT:

In [54]:
with open(SETTINGS_TXT, 'r') as f:
    data = f.readlines()
    wallet_addresses_list = [adress.replace('\n', '') for adress in data]

In [55]:
wallet_addresses_list

['0xA3D2e233658387701B92f0DA0cD422E314490C1F',
 '0xf02EF849af18e6BBa06024F8Ed4C56A2c5ea1556',
 '0x1902b720961c1E257d4A290726A5E8809903de0D',
 '0x7F29bAeE91eadbF7033b1B55536a063edD7242a8',
 '0xe6A1faF76e7b8281F66173e506AC7341f707FEBF',
 '0xc9524939551d5C97d611029A3C8150694396F537',
 '0x60378c4E2e56b7790839DA521E777B78aB1bfB5d',
 '0x065968Cfcf80d99b9695F36BdBa90B67B0B48505',
 '0x3309821aa89b1D42fe8EACff2FA56CC8c5FA06FE',
 '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390']

#### Отправка

Подключение:

In [56]:
fundingAPI = Funding.FundingAPI(API_KEY, SECRET_KEY, PASSPHRASE, False, '0')

Документация отправки активов с биржи https://www.okx.com/docs-v5/en/?python#funding-account-rest-api-withdrawal

ccy - название монеты<br> 
toAddr - адрес кошелька<br> 
amt - количество монет<br> 
fee - комиссия<br>
dest - внутренние трансферы или ончейн отправка, нам нужно ончейн параметр 4<br>
chain - название сети

Что происходит в скрипте:<br>
<br>
На каждой итерации цикла берется адрес кошелька из списка wallet_addresses_list,<br>
<br>
В sending_amount определяется количество монет для отправки - TOTAL_SENDING_AMOUNT делится на количество кошельков в списке и вычитается случайное число.<br>
<br>
В fundingAPI.withdrawal осуществляется тразакция, ответ записывается в api_response.<br>
<br>
Если в ответе 'code' равно 0, отправка прошла успешно, записывается отчет в xlsx. Если 'code' не равен нулю, при отправке возникла ошибка, все данные, код ошибки и расшифровка ошибки записывается в отчет xlsx.<br>
<br>
Скрипт засыпает на какое-то время, выбранное из диапазона.<br>
<br>
Если в цикле возникла ошибка, в блоке except записывается отчет с ошибкой.


In [ ]:
for address in wallet_addresses_list:
    try:    
        sending_amount = (TOTAL_SENDING_AMOUNT / len(wallet_addresses_list) - 
                          FEE -
                          random.uniform(TOTAL_SENDING_AMOUNT / MIN_VALUE_IN_RANGE_DIVIDER,
                                         TOTAL_SENDING_AMOUNT / MAX_VALUE_IN_RANGE_DIVIDER))
        
        print(f'Sending {sending_amount} {ASSET_NAME} to address {address} in chain {CHAIN_NAME}')

        api_response = fundingAPI.withdrawal(
            ccy=ASSET_NAME,
            toAddr=address,
            amt=sending_amount,
            fee=FEE,
            dest="4",
            chain=CHAIN_NAME
        )
        
        if api_response['code'] == '0':
            print('Sent')
            print('Creating report', '\n')
            report_table = open_table()
            create_report(report_table, address, ASSET_NAME, CHAIN_NAME, sending_amount, '-')
            
        elif api_response['code'] != '0':
            print('Didnt send')
            print('Creating report', '\n')
            error_code = api_response['code']
            error_message = api_response['msg']
            error_report_string = f'{error_code}-{error_message}'
            
            report_table = open_table()
            create_report(report_table, address, ASSET_NAME, CHAIN_NAME, sending_amount, error_report_string)

        time_sleep_value = random.choice(range(MIN_TIMESLEEP_BETWEEN_TRANSACTIONS, 
                                               MAX_TIMESLEEP_BETWEEN_TRANSACTIONS))
        print('Delay', time_sleep_value, '\n')

        time.sleep(time_sleep_value)
        
    except Exception as error:
        print('Didnt send')
        print('Creating report', '\n')
        report_table = open_table()
        create_report(report_table, '-', '-', '-', '-', error)

Если не хотите записывать отчет в таблицу, закоментируйте или удалите строки, начиная с 18 (if api_response['code'] == '0') по 32 (create_report...), а также закомментируйте 43 и 44 строки в блоке except (report_table...create_report...).

### Вариант 2 - настраиваемая отправка: разные монеты в разном количестве на кошельки<a class="anchor" id="6"></a>

Файл withdraw-from-okx-to-wallets-var-2.py

В этом варианте скрипту передается файл с настройками - xlsx или txt, в котором каждому кошельку, на который нужно отправить крипту, задается своя монета, количество монет, комиссия и сеть для отправки.

#### Библиотеки

In [57]:
import random
import time

import pandas as pd

import okx.Funding as Funding

#### Настройки

Настройки вашего аккаунта:

In [58]:
API_KEY = '8gb2d2f2-274b-455d-967e-9fdcb490f46c'
SECRET_KEY = 'D27A0AA2C55BC18B15055580921E56209'
PASSPHRASE = 'secret_phrase'

Название файла с настройками:

Excel:

In [59]:
SETTINGS_XLSX = 'sending_settings_var_2.xlsx'

Таблица содержит поля, в которых нужно будет указать соответствующие параметры:<br>
<br>
wallet_address - адрес кошелька<br>
asset_name - тикер монеты<br>
amount - количество, которое необходимо отправить<br>
fee - комиссия<br>
chain - название сети<br>

Как правильно писать тикер, название сети и корректное значение комисси можно узнать в полученных ранее данных, выгруженных в таблицу assets_settings.

или TXT:

In [60]:
SETTINGS_TXT = 'sending_settings_var_2.txt'

txt содержит все параметры для кошелька, перечисленные через запятую. Настройки для разных кошельков отделены точкой запятой.

#### Дополнительные настройки

Для задержки между транзакциями нужно задать минимальное время задержки и максимальное. Из диапазона этих значений будет выбираться случайное число.

Минимальная задержка (сек):

In [61]:
MIN_TIMESLEEP_BETWEEN_TRANSACTIONS = 240

Максимальная (сек):

In [62]:
MAX_TIMESLEEP_BETWEEN_TRANSACTIONS = 480

Название файла с отчетом xlsx, который используется в функции ниже:

In [63]:
REPORT_NAME = 'report_table_var_2.xlsx'

#### Функции для создания отчета

Такие же функции для создания отчета, которые использовались в первом примере скрипта.

excel_export записывает данные в excel. Принимает таблицу с данными (датафрейм), название будущего отчета:

In [64]:
def excel_export(table, table_name):
    writer_kernel = pd.ExcelWriter(table_name, engine='xlsxwriter')
    table.to_excel(writer_kernel, index=False)
    writer_kernel.close()

open_table открывает таблицу с отчетом, если таблицы нет, создает ее:

In [65]:
def open_table():
    try:
        table = pd.read_excel(REPORT_NAME)
        
        return table.to_dict('records')
    
    except:
        
        table_template = pd.DataFrame(index=[0])
        
        excel_export(table_template, REPORT_NAME)
        
        created_table = pd.read_excel(REPORT_NAME)
                
        return created_table.to_dict('records')

create_report принимает открытую таблицу, адрес кошелька, название монеты, сеть, отправленную сумму, ошибку, если она возникла, затем записывает данные в соответствующие поля, экспортирует xlsx:

In [66]:
def create_report(report_table, address, asset_name, chain, sent_amount, error):
        
    report_dict = dict()
    report_dict['address'] = address
    report_dict['asset_name'] = asset_name
    report_dict['chain'] = chain
    report_dict['sent_amount'] = sent_amount
    report_dict['error'] = error
        
    report_table.append(report_dict)
        
    result_table = pd.DataFrame.from_dict(report_table)
    
    result_table.dropna(inplace=True)
    
    excel_export(result_table, REPORT_NAME)

#### Загрузка файлов с настройками

Excel:

In [68]:
assets_settings_for_sending = pd.read_excel(SETTINGS_XLSX)

In [69]:
assets_settings_for_sending

,wallet_address,asset_name,amount,fee,chain
0,0xA3D2e233658387701B92f0DA0cD422E314490C1F,USDC,10.0000,0.1000,USDC-Arbitrum One (Bridged)
1,0xf02EF849af18e6BBa06024F8Ed4C56A2c5ea1556,USDT,10.0000,0.8000,USDT-Avalanche C-Chain
2,0x1902b720961c1E257d4A290726A5E8809903de0D,ETH,0.0035,0.0001,ETH-Arbitrum One
3,0x7F29bAeE91eadbF7033b1B55536a063edD7242a8,USDT,10.0000,0.1000,USDT-Optimism
4,0xe6A1faF76e7b8281F66173e506AC7341f707FEBF,USDC,10.0000,0.8000,USDC-Polygon
5,0xc9524939551d5C97d611029A3C8150694396F537,ETH,8.0000,0.0001,ETH-Optimism
6,0x60378c4E2e56b7790839DA521E777B78aB1bfB5d,ETH,7.0000,0.0001,ETH-Arbitrum One
7,0x065968Cfcf80d99b9695F36BdBa90B67B0B48505,AVAX,0.3000,0.0064,AVAX-Avalanche C-Chain
8,0x3309821aa89b1D42fe8EACff2FA56CC8c5FA06FE,ETH,0.0040,0.0003,ETH-zkSync Era
9,0xdf24026b19c60fCF9cD6145c237BD140EFe9f390,BNB,0.0200,0.0020,BNB-BSC


Для дальнейшей работы, конвертирую датафрейм в список словарей:

In [70]:
list_of_assets_settings_for_sending = assets_settings_for_sending.to_dict('records')

In [71]:
list_of_assets_settings_for_sending

[{'wallet_address': '0xA3D2e233658387701B92f0DA0cD422E314490C1F',
  'asset_name': 'USDC',
  'amount': 10.0,
  'fee': 0.1,
  'chain': 'USDC-Arbitrum One (Bridged)'},
 {'wallet_address': '0xf02EF849af18e6BBa06024F8Ed4C56A2c5ea1556',
  'asset_name': 'USDT',
  'amount': 10.0,
  'fee': 0.8,
  'chain': 'USDT-Avalanche C-Chain'},
 {'wallet_address': '0x1902b720961c1E257d4A290726A5E8809903de0D',
  'asset_name': 'ETH',
  'amount': 0.0035,
  'fee': 0.0001,
  'chain': 'ETH-Arbitrum One'},
 {'wallet_address': '0x7F29bAeE91eadbF7033b1B55536a063edD7242a8',
  'asset_name': 'USDT',
  'amount': 10.0,
  'fee': 0.1,
  'chain': 'USDT-Optimism'},
 {'wallet_address': '0xe6A1faF76e7b8281F66173e506AC7341f707FEBF',
  'asset_name': 'USDC',
  'amount': 10.0,
  'fee': 0.8,
  'chain': 'USDC-Polygon'},
 {'wallet_address': '0xc9524939551d5C97d611029A3C8150694396F537',
  'asset_name': 'ETH',
  'amount': 8.0,
  'fee': 0.0001,
  'chain': 'ETH-Optimism'},
 {'wallet_address': '0x60378c4E2e56b7790839DA521E777B78aB1bfB5d',

TXT:

<b>Обратите внимание, что каждый параметры для транзакции отделяется друг от друга запятой, поэтому не используйте запятую в десятичных числах, сипользуйте точку. Например 0.003, а не 0,003. Если будете использовать запятую, скрипт создаст два значения - 0 и 003.</b>

In [76]:
with open(SETTINGS_TXT, 'r') as file:
    lines = file.readlines()

list_of_assets_settings_for_sending = []
for line in lines:
    elements = line.strip().split(';')
    
    wallet_address, asset_name, amount, fee, chain_name = elements[0].split(',')
    data = {
        'wallet_address': wallet_address,
        'asset_name': asset_name,
        'amount': amount,
        'fee': fee,
        'chain_name': chain_name
    }

    list_of_assets_settings_for_sending.append(data)

In [77]:
list_of_assets_settings_for_sending

[{'wallet_address': '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390',
  'asset_name': 'USDC',
  'amount': '10',
  'fee': '0.1',
  'chain_name': 'USDC-Arbitrum One (Bridged)'},
 {'wallet_address': '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390',
  'asset_name': 'USDT',
  'amount': '10',
  'fee': '0.8',
  'chain_name': 'USDT-Avalanche C-Chain'},
 {'wallet_address': '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390',
  'asset_name': 'ETH',
  'amount': '0.0035',
  'fee': '0.0001',
  'chain_name': 'ETH-Arbitrum One'},
 {'wallet_address': '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390',
  'asset_name': 'USDT',
  'amount': '10',
  'fee': '0.1',
  'chain_name': 'USDT-Optimism'},
 {'wallet_address': '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390',
  'asset_name': 'USDC',
  'amount': '10',
  'fee': '0.8',
  'chain_name': 'USDC-Polygon'},
 {'wallet_address': '0xdf24026b19c60fCF9cD6145c237BD140EFe9f390',
  'asset_name': 'ETH',
  'amount': '8',
  'fee': '0.0001',
  'chain_name': 'ETH-Optimism'},
 {'wallet_address': '

#### Отправка

Подключение:

In [ ]:
fundingAPI = Funding.FundingAPI(API_KEY, SECRET_KEY, PASSPHRASE, False, '0')

Документация отправки активов с биржи https://www.okx.com/docs-v5/en/?python#funding-account-rest-api-withdrawal

ccy - название монеты<br> 
toAddr - адрес кошелька<br> 
amt - количество монет<br> 
fee - комиссия<br>
dest - внутренние трансферы или ончейн отправка, нам нужно ончейн параметр 4<br>
chain - название сети

Что происходит в скрипте:<br>
<br>
На каждой итерации цикла берется строка из списка словарей list_of_assets_settings_for_sending, из словарей определяются параметры и записываются в соответствующие переменные: wallet_address, asset_name, amount, fee, chain.<br>
<br>
В fundingAPI.withdrawal осуществляется тразакция, ответ записывается в api_response.<br>
<br>
Если в ответе 'code' равно 0, отправка прошла успешно, записывается отчет в xlsx. Если 'code' не равен нулю, при отправке возникла ошибка, все данные, код ошибки и расшифровка ошибки записывается в отчет xlsx.<br>
<br>
Скрипт засыпает на какое-то время, выбранное из диапазона.<br>
<br>
Если в цикле возникла ошибка, в блоке except записывается отчет с ошибкой.


In [ ]:
for row in list_of_assets_settings_for_sending:
    try:
        wallet_address, asset_name, amount, fee, chain = (row['wallet_address'], row['asset_name'], 
                                                          row['amount'], row['fee'], row['chain'])

        print(f'Sending {amount} {asset_name} to address {wallet_address} in chain {chain}')
        
        api_response = fundingAPI.withdrawal(
            ccy=asset_name,
            toAddr=wallet_address,
            amt=amount - fee,
            fee=fee,
            dest="4",
            chain=chain
        )
        
        if api_response['code'] == '0':
            print('Sent')
            print('Creating report', '\n')
            report_table = open_table()
            create_report(report_table, wallet_address, asset_name, chain, amount, '-')
            
        elif api_response['code'] != '0':
            print('Didnt send')
            print('Creating report', '\n')
            error_code = api_response['code']
            error_message = api_response['msg']
            error_report_string = f'{error_code}-{error_message}'
            
            report_table = open_table()
            create_report(report_table, wallet_address, asset_name, chain, amount, error_report_string)

        time_sleep_value = random.choice(range(MIN_TIMESLEEP_BETWEEN_TRANSACTIONS, 
                                               MAX_TIMESLEEP_BETWEEN_TRANSACTIONS))
        print('Delay', time_sleep_value, '\n')

        time.sleep(time_sleep_value)
        
    except Exception as error:
        print('Didnt send')
        print('Creating report', '\n')
        report_table = open_table()
        create_report(report_table, '-', '-', '-', '-', error)

Если не хотите записывать отчет в таблицу, закоментируйте или удалите строки, начиная с 17 (if api_response['code'] == '0') по 31 (create_report...), а также закомментируйте 42 и 43 строки в блоке except (report_table...create_report...).

## Часть 2 - отправка крипты с кошельков на саб-аккаунты Okx<a class="anchor" id="7"></a>

Чтобы не создавать связь между кошельками, нужно отправлять крипту на саб-аккаунты. Каждый кошелек соответствует адресу на саб-аккаунте. Так завещали старшие.

Не смог найти в документации api-метода, чтобы получить адреса для депозита на саб-акках. Поэтому придется взять адреса через интерфейс биржи.

Код взят из отличной статьи https://habr.com/ru/articles/674204/

### Вариант 1 - отправка нативных монет<a class="anchor" id="8"></a>

Файл withdraw-from-wallets-to-okx-var-1.py

В этом варианте скрипт получает файл с настройками и отправляет нативные монеты сети на адреса саб-акков okx. Настройки содержат:
- приватный ключ кошелька
- адрес okx
- название монеты
- название сети
- количество отправляемых монет
- url rpc сети

#### Библиотеки

In [78]:
import random
import time

from web3 import Web3

import pandas as pd

#### Настройки

Excel:

In [79]:
SETTINGS_XLSX = 'sending_from_wallets_settings_var_1.xlsx'

TXT:

In [80]:
SETTINGS_TXT = 'sending_from_wallets_settings_var_1.txt'

#### Дополнительные настройки

Для задержки между транзакциями нужно задать минимальное время задержки и максимальное. Из диапазона этих значений будет выбираться случайное число.

Минимальная задержка (сек):

In [81]:
MIN_TIMESLEEP_BETWEEN_TRANSACTIONS = 240

Максимальная (сек):

In [82]:
MAX_TIMESLEEP_BETWEEN_TRANSACTIONS = 480

Название файла с отчетом xlsx, который используется в функции ниже:

In [83]:
REPORT_NAME = 'report_table_var_sending_from_wallets_1.xlsx'

#### Функции для создания отчета

Такие же функции для создания отчета, которые использовались в первом примере скрипта.

excel_export записывает данные в excel. Принивает таблицу с данными (датафрейм), название будущего отчета:

In [84]:
def excel_export(table, table_name):
    writer_kernel = pd.ExcelWriter(table_name, engine='xlsxwriter')
    table.to_excel(writer_kernel, index=False)
    writer_kernel.close()

open_table открывает таблицу с отчетом, если таблицы нет, создает ее:

In [85]:
def open_table():
    try:
        table = pd.read_excel(REPORT_NAME)
        
        return table.to_dict('records')
    
    except:
        
        table_template = pd.DataFrame(index=[0])
        
        excel_export(table_template, REPORT_NAME)
        
        created_table = pd.read_excel(REPORT_NAME)
                
        return created_table.to_dict('records')

create_report принимает открытую таблицу, адрес кошелька, название монеты, сеть, отправленную сумму, hash транзакции, ошибку, если она возникла, затем записывает данные в соответствующие поля, экспортирует xlsx:

In [86]:
def create_report(report_table, wallet_address, okx_address, asset_name, chain, sent_amount, txn_hash, error):
        
    report_dict = dict()
    report_dict['wallet_address'] = wallet_address
    report_dict['okx_address'] = okx_address
    report_dict['asset_name'] = asset_name
    report_dict['chain'] = chain
    report_dict['sent_amount'] = sent_amount
    report_dict['txn_hash'] = txn_hash
    report_dict['error'] = error
        
    report_table.append(report_dict)
        
    result_table = pd.DataFrame.from_dict(report_table)
    
    result_table.dropna(inplace=True)
    
    excel_export(result_table, REPORT_NAME)

#### Загрузка файлов с настройками

Название таблицы Excel с настройками:

In [87]:
settings_for_sending_from_wallets = pd.read_excel(SETTINGS_XLSX)

In [88]:
settings_for_sending_from_wallets

,private_key,okx_sub_address,asset_name,chain_name,amount,chain_rpc
0,0x7500c937d5417c396e53125499889a8442a04379f77e...,0xf5f932dacbf7344ad813240f0104d1cc7dbb661,ETH,Abitrum,0.0035,https://arb-mainnet-public.unifra.io
1,0x20b07042e6441e554b30dfced0866462b7a7f37bb9d8...,0xf5f932dacbf7344ad813240f0104d1cc7dbb664,ETH,Optimism,0.1000,https://optimism.meowrpc.com
2,0x0186ef92357a66788fd9196557573cd22381c3c4acc7...,0xf5f932dacbf7344ad813240f0104d1cc7dbb665,ETH,Abitrum,0.7000,https://arb-mainnet-public.unifra.io
3,0x78fa3a27115057afd03b42c351deae01c97617a1c3c5...,0xf5f932dacbf7344ad813240f0104d1cc7dbb666,AVAX,Avalanche,0.3000,https://avalanche-c-chain.publicnode.com
4,0x3d6edd8823f6b6a8504e37b3c172d16470cfe0d4217e...,0xf5f932dacbf7344ad813240f0104d1cc7dbb667,ETH,Optimism,0.0040,https://optimism.meowrpc.com
5,0x1d9d73176373121295b0c6ebea1398f512bc72af5e8a...,0xf5f932dacbf7344ad813240f0104d1cc7dbb668,BNB,BSC,0.0200,https://bsc-dataseed1.bnbchain.org


In [89]:
list_of_settings_for_sending_from_wallets = settings_for_sending_from_wallets.to_dict('records')

In [90]:
list_of_settings_for_sending_from_wallets

[{'private_key': '0x7500c937d5417c396e53125499889a8442a04379f77e7133fb6e2fe729805b56',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb661',
  'asset_name': 'ETH',
  'chain_name': 'Abitrum',
  'amount': 0.0035,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0x20b07042e6441e554b30dfced0866462b7a7f37bb9d8b30e454ffcee69c007ce',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb664',
  'asset_name': 'ETH',
  'chain_name': 'Optimism',
  'amount': 0.1,
  'chain_rpc': 'https://optimism.meowrpc.com'},
 {'private_key': '0x0186ef92357a66788fd9196557573cd22381c3c4acc76c1376c65f08cbae64ae',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb665',
  'asset_name': 'ETH',
  'chain_name': 'Abitrum',
  'amount': 0.7,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0x78fa3a27115057afd03b42c351deae01c97617a1c3c5968aeba62e865006f359',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb666',
  'asset_name': 'AVAX',

или используйте TXT:

<b>Обратите внимание, что каждый параметры для транзакции отделяется друг от друга запятой, поэтому не используйте запятую в десятичных числах, сипользуйте точку. Например 0.003, а не 0,003. Если будете использовать запятую, скрипт создаст два значения - 0 и 003.</b>

In [91]:
with open(SETTINGS_TXT, 'r') as file:
    lines = file.readlines()

list_of_assets_settings_for_sending = []
for line in lines:
    elements = line.strip().split(';')
    
    private_key, okx_sub_address, asset_name, chain_name, amount, chain_rpc = elements[0].split(',')
    data = {
        'private_key': private_key,
        'okx_sub_address': okx_sub_address,
        'asset_name': asset_name,
        'chain_name': chain_name,
        'amount': amount,
        'chain_rpc': chain_rpc
    }

    list_of_assets_settings_for_sending.append(data)

In [92]:
list_of_settings_for_sending_from_wallets

[{'private_key': '0x7500c937d5417c396e53125499889a8442a04379f77e7133fb6e2fe729805b56',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb661',
  'asset_name': 'ETH',
  'chain_name': 'Abitrum',
  'amount': 0.0035,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0x20b07042e6441e554b30dfced0866462b7a7f37bb9d8b30e454ffcee69c007ce',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb664',
  'asset_name': 'ETH',
  'chain_name': 'Optimism',
  'amount': 0.1,
  'chain_rpc': 'https://optimism.meowrpc.com'},
 {'private_key': '0x0186ef92357a66788fd9196557573cd22381c3c4acc76c1376c65f08cbae64ae',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb665',
  'asset_name': 'ETH',
  'chain_name': 'Abitrum',
  'amount': 0.7,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0x78fa3a27115057afd03b42c351deae01c97617a1c3c5968aeba62e865006f359',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb666',
  'asset_name': 'AVAX',

#### Отправка

Что происходит в скрипте:<br>
<br>
На каждой итерации цикла из файла с настройками определяются все параметры для транзакции.<br>
<br>
Выполняется подключение по url rpc, которое указано в настройках.<br>
<br>
Из приватного ключа, который указан в настройках определяется публичный адрес.<br>
<br>
В amount_wei конвертируется количество монет в wei, указанное в настройках в ether.<br>
<br>
В gas_price определяется цена газа, в estimated_gas оценка лимита газа для отправки крипты, в fee расчитывается стоимость комиссии.<br>
<br>
В nonce определяется номер транзакции.<br>
<br>
В transaction_settings задаются все параметры будущей транзакции. Value рассчитывается из отправляемой суммы минус комиссия.<br>
<br>
В sent_amount конвертируется из wei в ether отправленная сумма. Для отчета.<br>
<br>
В signed_transaction подписывается транзакция, используется приватный ключ.<br>
<br>
send_raw_transaction отправляется подписанную транзакцию, hash записывается в transaction_hash.<br>
<br>
Записывается отчет.
<br>
<br>
В time_sleep_value определяется случайное время задержки. Скрипт засыпает.
<br>
<br>
Если срабатывает блок except, записывается данные транзакции и ошибка.


In [ ]:
for row in list_of_settings_for_sending_from_wallets:
    try:
        private_key, okx_sub_address, asset_name, chain_name, amount, chain_rpc = (row['private_key'],
                                                                                  row['okx_sub_address'],
                                                                                  row['asset_name'],
                                                                                  row['chain_name'],
                                                                                  row['amount'],
                                                                                  row['chain_rpc'])

        web3 = Web3(Web3.HTTPProvider(chain_rpc))

        from_address = web3.eth.account.from_key(private_key).address

        print(f'Sending {amount} {asset_name} on the {chain_name} network from address {from_address}')

        to_address = web3.to_checksum_address(okx_sub_address)

        print(f'To OKX address {to_address}')

        amount_wei = int(web3.to_wei(amount, 'ether'))

        gas_price = web3.eth.gas_price

        estimated_gas = web3.eth.estimate_gas({
            'to': to_address,
            'value': amount_wei,
        })

        fee = gas_price * estimated_gas

        nonce = web3.eth.get_transaction_count(from_address)

        transaction_settings = {
          'chainId': web3.eth.chain_id,
          'from': from_address,
          'to': to_address,
          'value': amount_wei - fee,
          'nonce': nonce, 
          'gasPrice': gas_price,
          'gas': estimated_gas,
        }

        sent_amount = float(web3.from_wei(amount_wei - fee, 'ether'))

        signed_transaction = web3.eth.account.sign_transaction(transaction_settings, private_key)

        transaction_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

        print('Sent')
        print('Txn hash:', transaction_hash.hex())

        print('Creating report')
        report_table = open_table()
        create_report(report_table, from_address, to_address, 
                      asset_name, chain_name, sent_amount, transaction_hash.hex(), '-')
        
        time_sleep_value = random.choice(range(MIN_TIMESLEEP_BETWEEN_TRANSACTIONS, 
                                               MAX_TIMESLEEP_BETWEEN_TRANSACTIONS))
        print('Delay', time_sleep_value, '\n')

        time.sleep(time_sleep_value)
        
    except Exception as error:
        print('Didnt send')
        print('Creating report')
        report_table = open_table()
        create_report(report_table, from_address, to_address, 
                      asset_name, chain_name, '-', '-', error)

### Вариант 2 - отправка токенов, например стейблов<a class="anchor" id="9"></a>

Файл withdraw-from-wallets-to-okx-var-2.py

В этом варианте скрипт получает файл с настройками и отправляет ERC20 токены на адреса саб-акков okx. Настройки содержат:
- приватный ключ кошелька
- адрес okx
- название монеты
- название сети
- адрес смарт-контракта
- количество отправляемых монет
- url rpc сети

#### Библиотеки

In [93]:
import random
import json
import time

from web3 import Web3
import pandas as pd

#### Настройки

Название json-файла, который содержит универсальный ABI для ERC20 токенов:

In [94]:
JSON_NAME = 'ERC20_ABI.json'

Название таблицы Excel с настройками:

In [95]:
SETTINGS_XLSX = 'sending_from_wallets_settings_var_2.xlsx'

txt аналог:

In [96]:
SETTINGS_TXT = 'sending_from_wallets_settings_var_2.txt'

#### Дополнительные настройки

Для задержки между транзакциями нужно задать минимальное время задержки и максимальное. Из диапазона этих значений будет выбираться случайное число.

Минимальная задержка (сек):

In [97]:
MIN_TIMESLEEP_BETWEEN_TRANSACTIONS = 240

Максимальная (сек):

In [98]:
MAX_TIMESLEEP_BETWEEN_TRANSACTIONS = 480

Название файла с отчетом xlsx, который используется в функции ниже:

In [99]:
REPORT_NAME = 'report_table_var_sending_from_wallets_2.xlsx'

#### Функции для создания отчета

Такие же функции для создания отчета, которые использовались в первом примере скрипта.

excel_export записывает данные в excel. Принивает таблицу с данными (датафрейм), название будущего отчета:

In [100]:
def excel_export(table, table_name):
    writer_kernel = pd.ExcelWriter(table_name, engine='xlsxwriter')
    table.to_excel(writer_kernel, index=False)
    writer_kernel.close()

open_table открывает таблицу с отчетом, если таблицы нет, создает ее:

In [101]:
def open_table():
    try:
        table = pd.read_excel(REPORT_NAME)
        
        return table.to_dict('records')
    
    except:
        
        table_template = pd.DataFrame(index=[0])
        
        excel_export(table_template, REPORT_NAME)
        
        created_table = pd.read_excel(REPORT_NAME)
                
        return created_table.to_dict('records')

create_report принимает открытую таблицу, адрес кошелька, название монеты, сеть, отправленную сумму, hash транзакции, ошибку, если она возникла, затем записывает данные в соответствующие поля, экспортирует xlsx:

In [102]:
def create_report(report_table, wallet_address, okx_address, asset_name, chain, sent_amount, txn_hash, error):
        
    report_dict = dict()
    report_dict['wallet_address'] = wallet_address
    report_dict['okx_address'] = okx_address
    report_dict['asset_name'] = asset_name
    report_dict['chain'] = chain
    report_dict['sent_amount'] = sent_amount
    report_dict['txn_hash'] = txn_hash
    report_dict['error'] = error
        
    report_table.append(report_dict)
        
    result_table = pd.DataFrame.from_dict(report_table)
    
    result_table.dropna(inplace=True)
    
    excel_export(result_table, REPORT_NAME)

#### Загрузка файлов с настройками

Название таблицы Excel с настройками:

In [112]:
settings_for_sending_from_wallets = pd.read_excel(SETTINGS_XLSX)

In [113]:
settings_for_sending_from_wallets

,private_key,okx_sub_address,asset_name,chain_name,token_contract,amount,chain_rpc
0,0xf44cf0306ddf13cfe2626dc496091d506bac81f9f2d4...,0xf5f932dacbf7344ad813240f0104d1cc7dbb659,USDC,Abitrum,0xff970a61a04b1ca14834a43f5de4533ebddb5cc8,10,https://arb-mainnet-public.unifra.io
1,0xad94d98ba4de6cd8c2b16cc27c384f27ce1bd2a65fba...,0xf5f932dacbf7344ad813240f0104d1cc7dbb660,USDT,Abitrum,0xff970a61a04b1ca14834a43f5de4533ebddb5cc9,10,https://arb-mainnet-public.unifra.io
2,0x7500c937d5417c396e53125499889a8442a04379f77e...,0xf5f932dacbf7344ad813240f0104d1cc7dbb661,USDT,Abitrum,0xff970a61a04b1ca14834a43f5de4533ebddb5cc10,13,https://arb-mainnet-public.unifra.io
3,0xf39b399f1b03197ea7ce85cd74d10a4e8f46aa2acbf5...,0xf5f932dacbf7344ad813240f0104d1cc7dbb662,USDT,Optimism,0xff970a61a04b1ca14834a43f5de4533ebddb5cc11,10,https://optimism.meowrpc.com
4,0x07ee336d988a595ff297d8efaad838e317de4b259800...,0xf5f932dacbf7344ad813240f0104d1cc7dbb663,USDC,Optimism,0xff970a61a04b1ca14834a43f5de4533ebddb5cc12,10,https://optimism.meowrpc.com
5,0x20b07042e6441e554b30dfced0866462b7a7f37bb9d8...,0xf5f932dacbf7344ad813240f0104d1cc7dbb664,USDC,Optimism,0xff970a61a04b1ca14834a43f5de4533ebddb5cc13,8,https://optimism.meowrpc.com
6,0x0186ef92357a66788fd9196557573cd22381c3c4acc7...,0xf5f932dacbf7344ad813240f0104d1cc7dbb665,USDC,Abitrum,0xff970a61a04b1ca14834a43f5de4533ebddb5cc14,7,https://arb-mainnet-public.unifra.io
7,0x78fa3a27115057afd03b42c351deae01c97617a1c3c5...,0xf5f932dacbf7344ad813240f0104d1cc7dbb666,USDC,Avalanche,0xff970a61a04b1ca14834a43f5de4533ebddb5cc15,24,https://avalanche-c-chain.publicnode.com
8,0x3d6edd8823f6b6a8504e37b3c172d16470cfe0d4217e...,0xf5f932dacbf7344ad813240f0104d1cc7dbb667,USDC,Optimism,0xff970a61a04b1ca14834a43f5de4533ebddb5cc16,25,https://optimism.meowrpc.com
9,0x1d9d73176373121295b0c6ebea1398f512bc72af5e8a...,0xf5f932dacbf7344ad813240f0104d1cc7dbb668,BUSD,BSC,0xff970a61a04b1ca14834a43f5de4533ebddb5cc17,30,https://bsc-dataseed1.bnbchain.org


In [114]:
list_of_settings_for_sending_from_wallets = settings_for_sending_from_wallets.to_dict('records')

In [115]:
list_of_settings_for_sending_from_wallets

[{'private_key': '0xf44cf0306ddf13cfe2626dc496091d506bac81f9f2d4fa9365b5944173f90afe',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb659',
  'asset_name': 'USDC',
  'chain_name': 'Abitrum',
  'token_contract': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc8',
  'amount': 10,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0xad94d98ba4de6cd8c2b16cc27c384f27ce1bd2a65fba3dec69e10ce9c50d7c4a',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb660',
  'asset_name': 'USDT',
  'chain_name': 'Abitrum',
  'token_contract': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc9',
  'amount': 10,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0x7500c937d5417c396e53125499889a8442a04379f77e7133fb6e2fe729805b56',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb661',
  'asset_name': 'USDT',
  'chain_name': 'Abitrum',
  'token_contract': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc10',
  'amount': 13,
  'chain_rpc': 'https://arb-m

или используйте TXT:

<b>Обратите внимание, что каждый параметры для транзакции отделяется друг от друга запятой, поэтому не используйте запятую в десятичных числах, сипользуйте точку. Например 0.003, а не 0,003. Если будете использовать запятую, скрипт создаст два значения - 0 и 003.</b>

In [116]:
with open(SETTINGS_TXT, 'r') as file:
    lines = file.readlines()

list_of_assets_settings_for_sending = []
for line in lines:
    elements = line.strip().split(';')
    
    private_key, okx_sub_address, asset_name, chain_name, token_contract, amount, chain_rpc = elements[0].split(',')
    data = {
        'private_key': private_key,
        'okx_sub_address': okx_sub_address,
        'asset_name': asset_name,
        'chain_name': chain_name,
        'token_contract': token_contract,
        'amount': amount,
        'chain_rpc': chain_rpc
    }

    list_of_assets_settings_for_sending.append(data)

In [117]:
list_of_settings_for_sending_from_wallets

[{'private_key': '0xf44cf0306ddf13cfe2626dc496091d506bac81f9f2d4fa9365b5944173f90afe',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb659',
  'asset_name': 'USDC',
  'chain_name': 'Abitrum',
  'token_contract': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc8',
  'amount': 10,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0xad94d98ba4de6cd8c2b16cc27c384f27ce1bd2a65fba3dec69e10ce9c50d7c4a',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb660',
  'asset_name': 'USDT',
  'chain_name': 'Abitrum',
  'token_contract': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc9',
  'amount': 10,
  'chain_rpc': 'https://arb-mainnet-public.unifra.io'},
 {'private_key': '0x7500c937d5417c396e53125499889a8442a04379f77e7133fb6e2fe729805b56',
  'okx_sub_address': '0xf5f932dacbf7344ad813240f0104d1cc7dbb661',
  'asset_name': 'USDT',
  'chain_name': 'Abitrum',
  'token_contract': '0xff970a61a04b1ca14834a43f5de4533ebddb5cc10',
  'amount': 13,
  'chain_rpc': 'https://arb-m

Загрузка JSON:

In [118]:
with open(JSON_NAME, 'r') as file:
    abi_json = json.load(file)

In [119]:
abi_json

[{'inputs': [{'internalType': 'string', 'name': '_name', 'type': 'string'},
   {'internalType': 'string', 'name': '_symbol', 'type': 'string'},
   {'internalType': 'uint256', 'name': '_initialSupply', 'type': 'uint256'}],
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'spender',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'value',
    'type': 'uint256'}],
  'name': 'Approval',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'from',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'to',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'value',
    'type': 'uint256'}],
  'name': 'Transfer',
  'ty

#### Отправка

Что происходит в скрипте:<br>
<br>
На каждой итерации цикла из файла с настройками определяются все параметры для транзакции.<br>
<br>
Выполняется подключение по url rpc, которое указано в настройках.<br>
<br>
Из приватного ключа, который указан в настройках определяется публичный адрес.<br>
<br>
В token_contract происходит инициализация указанного в настройках контракта.
<br>
<br>
В token_decimals определяется количество знаков для токена - формат, в котором он хранится в какой-либо сети.
<br>
<br>
В sending_amount записывается количество токенов для отправки в соответствующем формате.
<br>
<br>
В transaction_settings задаются все параметры будущей транзакции.<br>
<br>
В transaction создается транзакция.<br>
<br>
В signed_transaction подписывается транзакция, используется приватный ключ.<br>
<br>
send_raw_transaction отправляется подписанную транзакцию, hash записывается в transaction_hash.<br>
<br>
В sent_amount отправленная сумма конвертируется из wei в ether для отчета<br>
<br>
Записывается отчет.
<br>
<br>
В time_sleep_value определяется случайное время задержки. Скрипт засыпает.
<br>
<br>
Если срабатывает блок except, записывается данные транзакции и ошибка.


In [ ]:
for row in list_of_settings_for_sending_from_wallets:
    try:
        private_key, okx_sub_address, asset_name, chain_name, contract, amount, chain_rpc = (row['private_key'],
                                                                                            row['okx_sub_address'],
                                                                                            row['asset_name'],
                                                                                            row['chain_name'],
                                                                                            row['token_contract'],
                                                                                            row['amount'],
                                                                                            row['chain_rpc'])
        web3 = Web3(Web3.HTTPProvider(chain_rpc))

        from_address = web3.eth.account.from_key(private_key).address

        print(f'Sending {amount} {asset_name} on the {chain_name} network from address {from_address}')

        to_address = web3.to_checksum_address(okx_sub_address)

        print(f'To OKX address {to_address}')

        token_contract = web3.eth.contract(web3.to_checksum_address(contract), abi=abi_json)

        token_decimals = token_contract.functions.decimals().call()

        sending_amount = int(amount * 10**token_decimals)
                                                
        transaction_settings = {
            'chainId': web3.eth.chain_id,
            'from': from_address,
            'gasPrice': web3.eth.gas_price,
            'nonce': web3.eth.get_transaction_count(from_address)
        }
        
        transaction = token_contract.functions.transfer(to_address,
                                                         sending_amount).build_transaction(transaction_settings)
                
        signed_transaction = web3.eth.account.sign_transaction(transaction, private_key)
        
        transaction_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)
        
        sent_amount = float(web3.from_wei(sending_amount, 'ether'))
        
        print('Sent')
        print('Txn hash:', transaction_hash.hex())

        print('Creating report')
        report_table = open_table()
        create_report(report_table, from_address, to_address, 
                      asset_name, chain_name, sent_amount, transaction_hash.hex(), '-')

        time_sleep_value = random.choice(range(MIN_TIMESLEEP_BETWEEN_TRANSACTIONS, 
                                               MAX_TIMESLEEP_BETWEEN_TRANSACTIONS))
        print('Delay', time_sleep_value, '\n')

        time.sleep(time_sleep_value)

    except Exception as error:
        print('Didnt send')
        print('Creating report', '\n')
        print(error, '\n')
        report_table = open_table()
        create_report(report_table, from_address, to_address, 
                      asset_name, chain_name, '-', '-', error)